In [1]:
import pandas as pd
import numpy as np



In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import numpy as np
import re
import requests


list_of_roles = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM',
                 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 
                 'CB', 'RCB', 'RB']

fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy',
              'Short Passing', 'Volleys', 'Dribbling', 'Curve',
              'Free Kick Accuracy', 'Long Passing', 'Ball Control',
              'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
              'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
              'Long Shots', 'Aggression', 'Interceptions', 'Positioning',
              'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle',
              'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking',
              'GK Positioning', 'GK Reflexes']

cols =['full_name', 'player_id', 'age', 'rating', 'potential', 'club', 'club_joined', 
       'contract_valid', 'value', 'wage', 'release_clause', 'preff_foot',
       'Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball Control',
       'Composure', 'Crossing', 'Curve', 'DOB', 'Dribbling', 'Finishing',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Heading Accuracy', 'Interceptions', 'Jumping',
       'Long Passing', 'Long Shots', 'Marking', 'Penalties', 'Positioning',
       'Reactions', 'Short Passing', 'Shot Power', 'Sliding Tackle',
       'Sprint Speed', 'Stamina', 'Standing Tackle', 'Strength', 'Vision',
       'Volleys', 'club_jersey', 'club_pos',
       'country', 'country_jersey', 'country_pos',
       'country_rating','height', 'international_rep', 
       'traits', 'weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM',
                 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 
                 'CB', 'RCB', 'RB']
# We will create a list of url that gives all the pages of the players from sofifa
# Start and end should be between 0 and 226

def url_list_generator(start, end):
    
    l =[]
    
    url_list = []
    
    for x in range(226):
        l.append(x*80)
        
    for i in l:
        url_list.append('https://sofifa.com/players?showCol=jt&col=oa&sort=desc&offset={}'.format(i))
        
    url_list[0] ='https://sofifa.com/players'
    
    return url_list[start:end]

# soup_maker will generate the soup obj of a given url and the corresponding webpage

def soup_maker(url):
    
    r = requests.get(url)
    
    markup = r.content
    
    soup = bs(markup, 'lxml')
    
    return soup

# find_top_players will extract the links of  each 80 players

def find_top_players(soup):
    
    ## soup : Comes from soup_maker function. 
    ## soup object of a webpage like -- https://sofifa.com/players?offset=80
    ## This function returns a list of url's of player websites each url is a string type
    
    player_id = []
    
    table = soup.findAll('table', {'class': 'table-hover'})
    
    tbody = table[0].find('tbody')
    
    all_a = tbody.find_all('a')
    
    for player in all_a:
        
        if  player['href'].startswith('/player/'):
            player_id.append(player['href'])
            
    return ['http://sofifa.com' + player for player in player_id]

# find_player_info extract the most basic info for a player.

    
def find_player_info(soup):
    
    ## Here soup comes from soup_maker(url). url will be a player's webpage link.
    ## Return a dictionary that has the basic features of a player like name, id, dob, height and weight.
    
    player_data = {}
    
    player_data['full_name'] = soup.find('h1').text.split('(')[0]
   
    player_data['player_id'] = int(re.findall('\d\S\d*' ,soup.find('h1').text.split('(')[1])[0])
    
    player_data['role'] = soup.find('div', {'class': 'info'}).find_all('span', {'class':'pos'})[0].text
    
    # meta (Age -- (Apr 4, 1987) 170cm 75KG)
    meta = soup.find('div', {'class':'meta'}).contents[-1]
    
    # Create pattern to matcht () and numbers later in meta
    # pattern.findall(meta) = ['(Jul 1, 1989)']
    # pattern_digit.findall(meta) = ['30', '1', '1989', '170', '85']
    
    pattern = re.compile(r'\(.+\)')
    
    pattern_digit = re.compile(r'\d+')
    
    player_data['DOB'] = pattern.findall(meta)[0].replace(',','').lstrip('(').rstrip(')')
    
    player_data['age'] = int(pattern_digit.findall(meta)[0])
    
    player_data['height'] = int(pattern_digit.findall(meta)[3])
    
    player_data['weight'] = int(pattern_digit.findall(meta)[4])
    
    return player_data

# for url in ten_player:
#     soup = soup_maker(url)
#     info = soup.find('div', {'class': 'stats'}).find_all('div', {'class':'column'})

def find_player_stats(info):
    
    player_data = {}
    
    player_data['rating'] = int(info[0].span.text)
    
    player_data['potential'] = int( info[1].span.text.lstrip('p'))
    
    player_data['value'] = info[2].span.text.lstrip('€')
    
    player_data['wage'] = info[3].span.text.lstrip('€')
    
    return player_data

# for url in ten_player:
#     soup = soup_maker(url)

#     player_soup_box = soup.find('div', {'class':'player'}) \
#                                 .find('div', {'class':'card'}) \
#                                 .find('div',{'class': 'columns'}) \
#                                 .find_all('div',{'class': 'column'})
    
def find_player_secondary_info(player_soup_box, soup):

    player_data = {}

    player_data['preff_foot'] = player_soup_box[0]\
                                    .find_all('li')[0]\
                                    .text.strip('\nPreferred Foot')\
                                    .rstrip()

    player_data['international_rep'] = int(player_soup_box[0]\
                                                .find_all('li')[1].text\
                                                .strip('\nInternational Reputation')\
                                                .rstrip())
    
    if soup.find('label', text='Joined'):
        player_data['release_clause'] = player_soup_box[0]\
                                            .find('ul')\
                                            .find_all('li')[-1]\
                                            .contents[-2]\
                                            .text\
                                            .lstrip('€')


    player_data['club'] = player_soup_box[2]\
                             .find('ul')\
                             .find_all('li')[0]\
                             .text.lstrip()

    player_data['club_pos'] = player_soup_box[2]\
                                .find('ul')\
                                .find_all('li')[2]\
                                .contents[2]\
                                .text

    player_data['club_jersey'] = int(player_soup_box[2]\
                                         .find('ul')\
                                         .find_all('li')[3]\
                                         .contents[1]\
                                         .rstrip())

    if soup.find('label', text='Joined'):
        player_data['club_joined'] = player_soup_box[2]\
                                            .find('ul')\
                                            .find_all('li')[4]\
                                            .contents[2]\
                                            .rstrip()
        
    if soup.find('label', text = 'Loaned From'):
        player_data['club_joined'] = 'Loaned From ' + player_soup_box[2]\
                                                        .find('ul')\
                                                        .find_all('li')[4]\
                                                        .contents[2]\
                                                        .text 

    player_data['contract_valid'] = player_soup_box[2]\
                                            .find('ul')\
                                            .find_all('li')[5]\
                                            .contents[2]\
                                            .rstrip()

    if len(player_soup_box) > 3:
            player_data['country'] = player_soup_box[3]\
                                            .find('ul')\
                                            .find_all('li')[0]\
                                            .a\
                                            .text
                            
            player_data['country_rating'] = int(player_soup_box[3]\
                                                    .find('ul')\
                                                    .find_all('li')[1]\
                                                    .contents[0]\
                                                    .text)
            
            player_data['country_pos'] = player_soup_box[3]\
                                            .find('ul')\
                                            .find_all('li')[2]\
                                            .contents[2]\
                                            .text
                            
            player_data['country_jersey']= int(player_soup_box[3]\
                                                   .find('ul')\
                                                   .find_all('li')[3]
                                                   .contents[1]\
                                                    .rstrip())
            
    return player_data

def player_roles(soup):
    
    player_data = {}
    
    pattern = re.compile(r'\d+')
    
    for i in range(0,6):
        
        for div in  soup[i].find_all('div', {'class':'column'}):
            
            for roles in list_of_roles:
                
                if roles in div.text:
                    player_data[roles] = pattern.findall(div.contents[-1])[0]
                    
    return player_data

def player_fifa_info(fifa_info):
    
    player_data = {}
    
    col_list = []
    
    for i in range(0,7):
        col_list.extend(fifa_info[i].find('div').find_all('li'))
        
    pattern_digit = re.compile(r'\d+')
    
    for stats in fifa_stats:
        
        for li in col_list:
            
            if stats in li.text:
                player_data[stats] = int(pattern_digit.findall(li.text)[0])
                
    return player_data

def player_traits_info(soup):
    
    player_traits_list = []
    
    player_data = {}
    
    for li in soup:
        player_traits_list.append(li.text)
        
    player_data['traits'] = player_traits_list
    
    return player_data

def player_all_details(url):
    
    all_details = {}
    
    soup = soup_maker(url)

    all_details.update(find_player_info(soup))

    info = soup.find('div', {'class': 'stats'})\
                   .find_all('div', {'class':'column'})

    all_details.update(find_player_stats(info))

    player_soup_box = soup.find('div', {'class':'player'}).find('div', {'class':'card'})\
                    .find('div',{'class': 'columns'})\
                    .find_all('div',{'class': 'column'})

    all_details.update(find_player_secondary_info(player_soup_box, soup))

    fifa_info_soup = soup.find_all('div', {'class': 'column col-4 mb-20'})

    all_details.update(player_fifa_info(fifa_info_soup))
    
    overall_rating_soup = soup.find('h5', text = 'Real Overall Rating')
    
    if overall_rating_soup:
        
        s = soup.find_all('div', {'class':'toast mb-20'})[1].find_all('div',{'class':'columns'})
        
        all_details.update(player_roles(s))
        
    if soup.find('h5', text = 'Traits'):
        
        traits_soup = soup.find('ul', {'class': 'pl mb-20'}).find_all('li')
        
        all_details.update(player_traits_info(traits_soup))
    
    
        
    return all_details

def data_frame_generator(start, end):
    player_url_list = []
    pages = url_list_generator(start,end)
    df = pd.DataFrame()
    for main_page_url in pages:
        soup = soup_maker(main_page_url)
        player_url_list = find_top_players(soup)
        for i, url in enumerate(player_url_list):
            try:
                df = df.append(pd.DataFrame.from_dict(player_all_details(url), orient='index').T, ignore_index= True)
                df.to_csv('player_list_226.csv')
                if i%40 == 0:
                    print('En son kaydedilen url: {}'.format(url))
                    print('En son kaydedilen main_page: {}'.format(main_page_url))

            except:
                print('Hata for the url: {}'.format(url))
                print('Hata main_page: {}'.format(main_page_url))
    return df



def find_player_nationality(url):

        ## Here soup comes from soup_maker(url). url will be a player's webpage link.
        ## Return a dictionary that has the basic features of a player like name, id, dob, height and weight.
        soup = soup_maker(url)
        player_data = {}

        player_data['player_id'] = int(re.findall('\d\S\d*' ,soup.find('h1').text.split('(')[1])[0])

        player_data['nationality'] = soup.find_all('div', {'class':'meta'})[0].a.get('title')

        return player_data
    
def nationality_generator(start, end):
    player_url_list = []
    pages = url_list_generator(start,end)
    df = pd.DataFrame()
    for main_page_url in pages:
            soup = soup_maker(main_page_url)
            player_url_list = find_top_players(soup)
            for i, url in enumerate(player_url_list):
                try:
                    df = df.append(pd.DataFrame.from_dict(find_player_nationality(url), orient='index').T, ignore_index= True)
                    df.to_csv('player_nationality_26.csv')
                    if i%40 == 0:
                        print('En son kaydedilen url: {}'.format(url))
                        print('En son kaydedilen main_page: {}'.format(main_page_url))

                except:
                    print('Hata for the url: {}'.format(url))
                    print('Hata main_page: {}'.format(main_page_url))
    return df


In [ ]:
nationality_generator(26,50)